In [1]:
%pip install numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


### Суша море 

In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [12]:
def load_image(path):
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return image

def preprocess(image):
    denoised = cv2.medianBlur(image, 5)
    denoised = cv2.bilateralFilter(denoised, d=9, sigmaColor=75, sigmaSpace=75)
    return denoised

def land_water_segmentation(image):
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if np.mean(image[binary == 255]) < np.mean(image[binary == 0]):
        binary = cv2.bitwise_not(binary)
    return binary

def extract_land_objects(mask, min_area=100):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objects = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h > min_area:
            objects.append((x, y, x + w, y + h))
    return objects

def extract_water_contours(mask):
    water_mask = cv2.bitwise_not(mask)
    kernel = np.ones((3, 3), np.uint8)
    water_mask = cv2.morphologyEx(water_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    contours, _ = cv2.findContours(water_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_water_contours(image, contours, output_path=None):
    vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(vis, contours, -1, (255, 0, 0), 2)  # синие контуры
    if output_path:
        cv2.imwrite(output_path, vis)
    else:
        plt.imshow(vis[..., ::-1])
        plt.title("Контуры воды")
        plt.axis('off')
        plt.show()

def visualize(original, mask, boxes=None):
    image_color = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)
    if boxes:
        for x1, y1, x2, y2 in boxes:
            cv2.rectangle(image_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.figure(figsize=(16, 6))
    plt.subplot(1, 2, 1)
    plt.title("Маска суши")
    plt.imshow(mask, cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title("Обнаруженные объекты")
    plt.imshow(image_color[..., ::-1])
    plt.show()

### Второй вариант

In [22]:
def load_image(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)

def preprocess(image):
    denoised = cv2.medianBlur(image, 5)
    denoised = cv2.bilateralFilter(denoised, d=9, sigmaColor=75, sigmaSpace=75)
    return denoised

def segment_objects_then_water(image):
    # Текстура через Лапласиан
    lap = cv2.Laplacian(image, cv2.CV_64F)
    texture = cv2.convertScaleAbs(lap)

    # Статистика по изображению
    mean_val = np.mean(image)
    std_val = np.std(image)

    # Адаптивные пороги
    bright_thresh = mean_val + 0.8 * std_val   # для объектов
    dark_thresh = mean_val - 0.6 * std_val     # для воды
    mid_low = mean_val - 0.2 * std_val         # почва от
    mid_high = mean_val + 0.2 * std_val        # почва до

    # Суша (объекты): высокая текстура или выше порога яркости
    object_mask = np.logical_or(
        texture > 15,
        image > bright_thresh
    ).astype(np.uint8) * 255

    # Почва: умеренная яркость + небольшая текстура
    soil_mask = np.logical_and.reduce((
        image > mid_low,
        image < mid_high,
        texture > 10
    )).astype(np.uint8) * 255

    # Вода: низкая яркость + низкая текстура + не объект
    water_mask = np.logical_and.reduce((
        image < dark_thresh,
        texture < 10,
        object_mask == 0
    )).astype(np.uint8) * 255

    return object_mask, soil_mask, water_mask


def extract_land_objects(mask, min_area=100):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objects = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h > min_area:
            objects.append((x, y, x + w, y + h))
    return objects

def extract_contours(mask):
    kernel = np.ones((3, 3), np.uint8)
    clean_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, _ = cv2.findContours(clean_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_combined_visual(image, land_boxes, water_contours, soil_contours, output_path=None):
    vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    for x1, y1, x2, y2 in land_boxes:
        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)  # объекты: зелёный

    cv2.drawContours(vis, water_contours, -1, (255, 0, 0), 2)  # вода: синий
    cv2.drawContours(vis, soil_contours, -1, (255, 165, 0), 2)  # почва: оранжевый

    if output_path:
        cv2.imwrite(output_path, vis)
    else:
        plt.imshow(vis[..., ::-1])
        plt.title("Суша (зел), Вода (син), Почва (оранж)")
        plt.axis('off')
        plt.show()

def process_image(image_path, min_area=100):
    image = load_image(image_path)
    preprocessed = preprocess(image)
    object_mask, soil_mask, water_mask = segment_objects_then_water(preprocessed)

    land_boxes = extract_land_objects(object_mask, min_area)
    water_contours = extract_contours(water_mask)
    soil_contours = extract_contours(soil_mask)

    return image, water_mask, soil_mask, object_mask, land_boxes, water_contours, soil_contours

def process_folder(input_folder, output_folder, min_area=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            (image, water_mask, soil_mask, object_mask, 
             land_boxes, water_contours, soil_contours) = process_image(image_path, min_area)

            name = os.path.splitext(filename)[0]

            # Сохраняем маски
            cv2.imwrite(os.path.join(output_folder, f"{name}_water.png"), water_mask)
            cv2.imwrite(os.path.join(output_folder, f"{name}_soil.png"), soil_mask)
            cv2.imwrite(os.path.join(output_folder, f"{name}_objects.png"), object_mask)

            # Общая визуализация
            combined_output_path = os.path.join(output_folder, f"{name}_combined.png")
            draw_combined_visual(image, land_boxes, water_contours, soil_contours, combined_output_path)

            print(f"Processed {filename} → saved all outputs to {output_folder}")


### Вода плюс суша нарисовать вместе.

In [13]:
def draw_combined_visual(image, land_boxes, water_contours, output_path=None):
    vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Отрисовываем сушу (bounding boxes — зелёные)
    for x1, y1, x2, y2 in land_boxes:
        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Отрисовываем воду (контуры — синие)
    cv2.drawContours(vis, water_contours, -1, (255, 0, 0), 2)

    if output_path:
        cv2.imwrite(output_path, vis)
    else:
        plt.imshow(vis[..., ::-1])
        plt.title("Суша (зелёный) и вода (синий)")
        plt.axis('off')
        plt.show()


In [10]:
def process_image(image_path, min_area=100):
    image = load_image(image_path)
    preprocessed = preprocess(image)
    mask = land_water_segmentation(preprocessed)
    boxes = extract_land_objects(mask, min_area)
    water_contours = extract_water_contours(mask)
    return image, mask, boxes, water_contours

def process_folder(input_folder, output_folder, min_area=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            original, mask, boxes, water_contours = process_image(image_path, min_area)

            # Сохраняем маску суши
            mask_output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_mask.png")
            cv2.imwrite(mask_output_path, mask)

            # Сохраняем изображение с сушей (зелёные bbox'ы)
            vis_image = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)
            for x1, y1, x2, y2 in boxes:
                cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            vis_output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.png")
            cv2.imwrite(vis_output_path, vis_image)

            # Сохраняем изображение с контурами воды (синие линии)
            water_output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_water.png")
            draw_water_contours(original, water_contours, water_output_path)

            print(f"Processed {filename}: saved mask, land and water contours to output folder.")


### Отрисовка воды вместе с сушей на одном рисунке

In [14]:
def process_image(image_path, min_area=100):
    image = load_image(image_path)
    preprocessed = preprocess(image)
    mask = land_water_segmentation(preprocessed)
    boxes = extract_land_objects(mask, min_area)
    water_contours = extract_water_contours(mask)
    return image, mask, boxes, water_contours

def process_folder(input_folder, output_folder, min_area=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(input_folder, filename)
            original, mask, boxes, water_contours = process_image(image_path, min_area)

            # Сохраняем маску суши
            mask_output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_mask.png")
            cv2.imwrite(mask_output_path, mask)

            combined_output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_combined.png")
            draw_combined_visual(original, boxes, water_contours, combined_output_path)


            print(f"Processed {filename}: saved mask, land and water contours to output folder.")

In [23]:
input_folder = "images"
output_folder = "output"
process_folder(input_folder, output_folder)

Processed 8.jpg → saved all outputs to output
Processed 9.jpg → saved all outputs to output
Processed 4.jpg → saved all outputs to output
Processed 5.jpg → saved all outputs to output
Processed 7.jpg → saved all outputs to output
Processed 6.jpg → saved all outputs to output
Processed 2.jpg → saved all outputs to output
Processed 3.jpg → saved all outputs to output
Processed 1.jpg → saved all outputs to output
